In [1]:
from langchain_cohere import ChatCohere
from langgraph.graph import StateGraph,START,END
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict

In [2]:
class JokeState(TypedDict):
    topic:str
    jokes:str
    explanation:str

In [3]:
load_dotenv()
llm=ChatCohere(model="command-r-plus-08-2024")

In [4]:
def Gen_joke(state:JokeState)->dict:
    prompt=f"Write a joke on the given topic,Here is the topic:{state["topic"]}"
    response=llm.invoke(prompt)
    return {"jokes":response.content}

def Exp_joke(state:JokeState)->dict:
    prompt=f"Explain this joke:{state["jokes"]}"
    response=llm.invoke(prompt)
    return {"explanation":response.content}

In [5]:
graph=StateGraph(JokeState)

graph.add_node("generate_joke",Gen_joke)
graph.add_node("explain_joke",Exp_joke)

graph.add_edge(START,"generate_joke")
graph.add_edge("generate_joke","explain_joke")
graph.add_edge("explain_joke",END)

checkpointer=InMemorySaver()

workflow=graph.compile(checkpointer=checkpointer)

In [6]:
config1={"configurable":{"thread_id":"1"}}
workflow.invoke({"topic":"Cat"},config=config1)

{'topic': 'Cat',
 'jokes': 'Why did the cat join the Red Cross?\nBecause she wanted to be a first-aid kit!',
 'explanation': 'This joke is a play on words and a pun based on the homophone (words that sound the same but have different meanings and spellings) of "first aid" and "first aid kit."\n\nThe joke suggests that the cat wanted to join the Red Cross, a humanitarian organization known for providing medical assistance and disaster relief, because it wanted to become a "first-aid kit." In this context, the cat is equating itself with a physical kit containing medical supplies used for rendering initial treatment in case of an emergency.\n\nThe humor lies in the unexpected twist of the cat\'s motivation for joining the Red Cross, as it is a clever wordplay that combines the idea of helping others (first aid) with the concept of a physical kit containing medical supplies. It is a lighthearted and whimsical take on the noble work of the Red Cross and its volunteers.'}

In [7]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Cat', 'jokes': 'Why did the cat join the Red Cross?\nBecause she wanted to be a first-aid kit!', 'explanation': 'This joke is a play on words and a pun based on the homophone (words that sound the same but have different meanings and spellings) of "first aid" and "first aid kit."\n\nThe joke suggests that the cat wanted to join the Red Cross, a humanitarian organization known for providing medical assistance and disaster relief, because it wanted to become a "first-aid kit." In this context, the cat is equating itself with a physical kit containing medical supplies used for rendering initial treatment in case of an emergency.\n\nThe humor lies in the unexpected twist of the cat\'s motivation for joining the Red Cross, as it is a clever wordplay that combines the idea of helping others (first aid) with the concept of a physical kit containing medical supplies. It is a lighthearted and whimsical take on the noble work of the Red Cross and its volunteers.'}

In [8]:
# Here we can see the state saved at every checkpoint(after every superstep)
#start from below 

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Cat', 'jokes': 'Why did the cat join the Red Cross?\nBecause she wanted to be a first-aid kit!', 'explanation': 'This joke is a play on words and a pun based on the homophone (words that sound the same but have different meanings and spellings) of "first aid" and "first aid kit."\n\nThe joke suggests that the cat wanted to join the Red Cross, a humanitarian organization known for providing medical assistance and disaster relief, because it wanted to become a "first-aid kit." In this context, the cat is equating itself with a physical kit containing medical supplies used for rendering initial treatment in case of an emergency.\n\nThe humor lies in the unexpected twist of the cat\'s motivation for joining the Red Cross, as it is a clever wordplay that combines the idea of helping others (first aid) with the concept of a physical kit containing medical supplies. It is a lighthearted and whimsical take on the noble work of the Red Cross and its volunteers.'

Now Time travel begin

Step1:Copy the checkpoint_id form the list of workflow.get_state_history,ex: 'checkpoint_id': '1f0d1378-bf22-6423-8000-e30edbaa6768'

In [9]:
workflow.get_state({"configurable":{"thread_id":1,'checkpoint_id': '1f0d1378-bf22-6423-8000-e30edbaa6768'}})

StateSnapshot(values={'topic': 'Cat'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0d1378-bf22-6423-8000-e30edbaa6768'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-12-04T17:34:54.979382+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d1378-bf1d-6ece-bfff-7dec8e4eadfc'}}, tasks=(PregelTask(id='aeae1504-1b56-6be2-2ece-fba97c1d1c24', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'jokes': 'Why did the cat join the Red Cross?\nBecause she wanted to be a first-aid kit!'}),), interrupts=())

In [10]:
workflow.invoke(None,{"configurable":{"thread_id":1,'checkpoint_id': '1f0d1378-bf22-6423-8000-e30edbaa6768'}})

{'topic': 'Cat',
 'jokes': 'Why did the cat sit on the computer?\n\nBecause it wanted to keep an eye on the mouse!',
 'explanation': 'This joke plays on a double meaning of the word "mouse."\n\nIn the context of a computer, a mouse is an input device used to control the cursor on the screen. It is a common peripheral device connected to computers. So, the joke suggests that the cat is interested in monitoring or \'keeping an eye on\' this computer mouse.\n\nHowever, the word "mouse" also refers to a small rodent, which is a common prey for cats in the natural world. Cats are known for their hunting instincts, and they often \'keep an eye on\' mice as potential prey.\n\nBy combining these two meanings, the joke creates a humorous scenario where the cat is sitting on the computer, not because of any technological interest, but due to its natural instinct to watch over a mouse, the rodent. It\'s a clever play on words that relies on the multiple interpretations of \'mouse\' to generate hu

As we can see here is the different joke now,in this way we can implement different logics to debug

In [11]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Cat', 'jokes': 'Why did the cat sit on the computer?\n\nBecause it wanted to keep an eye on the mouse!', 'explanation': 'This joke plays on a double meaning of the word "mouse."\n\nIn the context of a computer, a mouse is an input device used to control the cursor on the screen. It is a common peripheral device connected to computers. So, the joke suggests that the cat is interested in monitoring or \'keeping an eye on\' this computer mouse.\n\nHowever, the word "mouse" also refers to a small rodent, which is a common prey for cats in the natural world. Cats are known for their hunting instincts, and they often \'keep an eye on\' mice as potential prey.\n\nBy combining these two meanings, the joke creates a humorous scenario where the cat is sitting on the computer, not because of any technological interest, but due to its natural instinct to watch over a mouse, the rodent. It\'s a clever play on words that relies on the multiple interpretations of \'mo

As in above more 2 state is added because of re-run

NOW lets update state

In [17]:
#give the checkpoint_id of that place where we want to update any state value

workflow.update_state(
    {"configurable": {
        "thread_id": "1",
        "checkpoint_id": "1f0d1378-bf22-6423-8000-e30edbaa6768",
        "checkpoint_ns": ""  # Usually empty string for top-level checkpoints
    }},
    {"topic": "samosa"}
)

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0d139f-f6b5-617f-8001-6d95f04c3c5b'}}

In [18]:
list(workflow.get_state_history(config1)) 

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d139f-f6b5-617f-8001-6d95f04c3c5b'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-12-04T17:52:27.704972+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d1378-bf22-6423-8000-e30edbaa6768'}}, tasks=(PregelTask(id='6380e73c-fc77-37f9-8adb-cda8980010a7', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d139f-4b6a-65ac-8001-dfede4456e71'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-12-04T17:52:09.743710+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id

In [ ]:
#write the checkpoint id of that state from where we want to re-execute

workflow.invoke(None,{"configurable":{"thread_id":1,'checkpoint_id': '1f0d139f-f6b5-617f-8001-6d95f04c3c5b'}})

{'topic': 'samosa',
 'jokes': 'Why did the samosa feel lonely?\nBecause it was a single piece and its friends were in pairs!',
 'explanation': 'This joke is a play on words and a pun based on the concept of "samosa" and the idea of being "single" or "in pairs."\n\nA samosa is a popular South Asian snack consisting of a fried or baked pastry with a savory filling, often in a triangular shape. In the context of the joke, the samosa is personified and described as feeling lonely.\n\nThe humor lies in the fact that the samosa is a "single piece," which is a reference to its individual, standalone nature. At the same time, its "friends" are described as being "in pairs," indicating that they come in sets of two. This creates a contrast between the solitary samosa and its paired companions, making the samosa\'s loneliness more comical.\n\nThe joke relies on the audience\'s understanding of the typical serving size of samosas, which are often served in pairs or multiples, making a single samo

In [21]:
list(workflow.get_state_history(config1)) 

[StateSnapshot(values={'topic': 'samosa', 'jokes': 'Why did the samosa feel lonely?\nBecause it was a single piece and its friends were in pairs!', 'explanation': 'This joke is a play on words and a pun based on the concept of "samosa" and the idea of being "single" or "in pairs."\n\nA samosa is a popular South Asian snack consisting of a fried or baked pastry with a savory filling, often in a triangular shape. In the context of the joke, the samosa is personified and described as feeling lonely.\n\nThe humor lies in the fact that the samosa is a "single piece," which is a reference to its individual, standalone nature. At the same time, its "friends" are described as being "in pairs," indicating that they come in sets of two. This creates a contrast between the solitary samosa and its paired companions, making the samosa\'s loneliness more comical.\n\nThe joke relies on the audience\'s understanding of the typical serving size of samosas, which are often served in pairs or multiples, 